# Package

In [2]:
import pandas as pd
import numpy as np
#from sklearn.neighbors import NearestNeighbors
from numpy import array
from numpy.linalg import norm

# Functions


In [3]:
def movieId_by_urserId(userid):
    """
    Input
    -----
        userid:int
    Return
    -----
    a list of movie id that rated by given user"""
    return list(rating[rating['userId'] == userid]['movieId'])

In [4]:
def search_rating_user_id(userid,movieid):
    """
    Input
    ---
    userid:int, represent userid
    movieid:int, represent movieid (closest id that rated by given user)"""
    return rating[(rating['userId'] == userid) & (rating['movieId'] == movieid)]['rating']

# Data

In [10]:
#result = pyreadr.read_r('../output/mat_fac.RData') 
p = pd.read_csv('../output/p_5.csv')
#rmse = pd.read_csv('../data/rmse.csv')
q = pd.read_csv('../output/q_5.csv') #movie latent values
rating = pd.read_csv('../data/ml-latest-small/ratings.csv')

In [12]:
q

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,-0.055353,0.314352,-0.265129,0.179237,0.389532,-0.208534,-0.199514,-0.557164,-0.142145,...,0.269451,0.071936,-0.336207,-0.594160,-0.617343,-0.878814,-0.901080,-0.836995,-0.395947,0.428670
1,2,0.214946,0.092921,0.239253,0.645300,0.016529,-0.392982,-0.014915,0.801285,-0.370590,...,-0.018329,0.060680,0.220290,0.947834,-0.886434,0.887563,0.006915,-0.585194,0.230590,0.313764
2,3,-0.330567,0.053810,-0.241691,-0.768676,0.453557,-0.067485,-0.142218,-0.734632,0.367337,...,0.348406,-0.191861,-0.281676,0.851102,0.917738,-0.628555,0.227531,0.098740,0.993204,-0.282733
3,4,0.520175,0.288946,-0.372331,0.046895,-0.214070,0.160670,-0.160177,-0.402434,-0.256543,...,-0.068974,0.596462,0.076719,0.223086,0.546831,0.872518,-0.887726,0.867172,-0.587827,0.797158
4,5,0.080285,-0.119310,0.497353,0.222890,0.207967,-0.268237,0.022038,-0.678434,0.412121,...,0.379974,-0.731514,0.483033,0.498212,0.061833,-0.651405,-0.793610,-0.706674,0.776282,-0.267834
5,6,0.184848,-0.103753,-0.350213,0.730413,0.195897,-0.083429,-0.325278,0.502118,0.607048,...,0.587805,0.685013,0.191695,0.030694,0.694699,-0.129783,0.456441,-0.091622,-0.274341,-0.106040
6,7,-0.234019,0.121795,0.154626,0.450407,0.045235,-0.080219,-0.093114,0.659501,-0.214100,...,-0.181729,0.620711,0.365603,0.494817,0.920988,0.329884,0.081240,0.823976,0.270760,-0.314284
7,8,-0.035348,-0.280624,-0.223273,0.920257,-0.422110,0.161072,0.240275,0.296029,0.224397,...,0.810205,0.048380,0.247360,0.518878,0.201255,-0.226172,0.903357,0.824490,0.333399,0.734490
8,9,0.242049,-0.196567,0.372898,0.371786,-0.080092,-0.183619,0.076585,0.072422,0.350290,...,-0.727389,-0.958855,-0.147408,-0.135500,0.273554,0.869255,-0.679211,0.775252,0.372155,0.332197
9,10,0.081950,0.248989,-0.074776,0.728523,0.076675,-0.087251,0.068927,-0.183351,0.279782,...,0.375732,0.700993,-0.980784,0.840699,-0.080450,-0.675125,0.454055,-0.334257,0.104638,-0.740559


In [13]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [8]:
userId = list(set(rating['userId'])) #a list of user id 

movie_index = list(q.columns[1:]) #list of all movie index,str

#movie_index_int = [int(val) for val in movie_index] #int form of movie_index


userid = 1

movieid_user = movieId_by_urserId(userid) #list of movie ID that rated by given user(passed latent)









In [ ]:
movieid_user = movieId_by_urserId(userid) #list of movie ID that rated by given user

In [15]:
movie_index = list(q.columns[1:]) #list of all movie index,str
movie_index_int = [int(val) for val in movie_index] #int form of movie_index


# Vj_index = ['1'] # list of given movie index
# movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [ ]:
Vj_index = ['1'] # list of given movie index
movie_index_sub = [item for item in movie_index if item not in Vj_index] #list of movie indexs except the comparied one

In [ ]:
def 



In [ ]:
l_latent_movie = []
for i in movie_index_sub:
   # print(i,type(i)) 2，str
    numerator =array([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist]).dot(array([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    
    cosin_s = numerator/(norm([val
                  for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
                  for val in sublist])*norm([val
                  for sublist in np.asarray(q[[i]]).tolist()
                  for val in sublist]))
    l_latent_movie.append(cosin_s)
    
index_min = np.argmin(l_latent_movie) #find the index of min value

In [129]:
# l_latent_movie = []
# for i in movie_index_sub:
#    # print(i,type(i)) 2，str
#     numerator =array([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
    
#     cosin_s = numerator/(norm([val
#                   for sublist in np.asarray(q[[Vj_index[0]]]).tolist()
#                   for val in sublist])*norm([val
#                   for sublist in np.asarray(q[[i]]).tolist()
#                   for val in sublist]))
#     l_latent_movie.append(cosin_s)
    
# index_min = np.argmin(l_latent_movie) #find the index of min value

# # if int(Vj_index[0]) > index_min: 
# #     index_similar = index_min
# # else:
# #     index_similar = index_min+1 #+1 would be the real index of that similar movie
    
# # print(index_similar)

5125


# scrachwork

In [54]:
#q.as_matrix(columns=q.columns[1:2]).dot(q.as_matrix(columns=q.columns[2:3]))

In [123]:
l_latent_movie = []
movie_index = q.columns[1:]
# for i in range(len(q.columns[1:])):
#     print(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))
#     #j = i+1
#     l_latent_movie.append(array([val
#                   for sublist in np.asarray(q[[movie_index[i+1]]]).tolist()
#                   for val in sublist]).dot(array([val
#                   for sublist in np.asarray(q[[movie_index[i+2]]]).tolist()
#                   for val in sublist])))

In [4]:
rmse.head()

,Unnamed: 0,train_test,par,epoch,RMSE
0,1,Train,"f = 10 , lambda = 0.01",1,0.825492
1,2,Train,"f = 20 , lambda = 0.01",1,0.792257
2,3,Train,"f = 10 , lambda = 0.1",1,0.896950
3,4,Train,"f = 20 , lambda = 0.1",1,0.859586
4,5,Test,"f = 10 , lambda = 0.01",1,1.313171


In [8]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]

In [13]:
neigh = NearestNeighbors(2, 0.4,metric='cosine')
neigh.fit(samples)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=0.4)

In [14]:
#class sklearn.neighbors.NearestNeighbors(n_neighbors=5, radius=1.0, algorithm=’auto’, leaf_size=30,
#metric=’minkowski’, p=2, metric_params=None, n_jobs=None, **kwargs)

In [15]:
neigh.predict([0,0,5])

AttributeError: 'NearestNeighbors' object has no attribute 'predict'